In [1]:
import oracledb
from dotenv import load_dotenv
import os
from db_func import *
import pandas as pd 
import random
import uuid

In [2]:
load_dotenv('.env')

oracledb.init_oracle_client(lib_dir=r"C:\instant_client\instantclient_21_19")
conn = oracledb.connect(
    user=os.getenv('user'),
    password=os.getenv('password'),
    dsn=os.getenv('ORACLE_DSN')
)
cur = conn.cursor()

In [4]:
def privilege_validation(cursor, table_name, user_name):
    sql = f'''SELECT 
        CASE 
            WHEN NOT EXISTS (
                SELECT 1
                FROM DBA_TAB_PRIVS
                WHERE OWNER = 'TEAM8'
                AND TABLE_NAME = '{table_name}'
                AND GRANTEE = '{user_name}'
            )
            THEN 'NO_PRIVILEGE'
            
            ELSE 'ACCESSIBLE'
        END AS RESULT
    FROM dual'''

    cursor.execute(sql)
    return cursor.fetchone()[0]
    
privilege_validation(cur, 'patients', "TEAM8_ADMIN_USER")

'NO_PRIVILEGE'

In [38]:
llm_text, bind_dict = '''SELECT
            first_careunit AS icu_unit,
            ROUND(AVG(los), 2) AS avg_icu_los
        FROM team8.icustays
        GROUP BY first_careunit
        ORDER BY avg_icu_los DESC''', {}

In [43]:
table_info = make_table_from_sql(conn, llm_text, bind_dict, limit=20)
df = pd.DataFrame(table_info['rows'], columns= table_info['columns'])
df_reset = df.reset_index(drop=True)
df_reset

,ICU_UNIT,AVG_ICU_LOS
0,Neurology,28.25
1,Medicine,15.79
2,Surgery/Vascular/Intermediate,15.71
3,Surgery/Trauma,10.64
4,Intensive Care Unit (ICU),8.79
5,Neuro Intermediate,5.02
6,Neuro Surgical Intensive Care Unit (Neuro SICU),4.48
7,Neuro Stepdown,4.07
8,PACU,4.02
9,Surgical Intensive Care Unit (SICU),3.90


In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ICU_UNIT     17 non-null     object 
 1   AVG_ICU_LOS  17 non-null     float64
dtypes: float64(1), object(1)
memory usage: 400.0+ bytes


In [30]:
import random
from datetime import datetime

cur.execute('SELECT saved_query_id from SAVED_QUERY')
will_update = random.choices([i[0] for i in cur.fetchall()], k = 23)

def random_time_20260219_11():
    minute = random.randint(0, 59)
    second = random.randint(0, 59)

    return datetime(2026, 2, 19, 10, minute, second)


for i in will_update:
    random_time = random_time_20260219_11()
    sql = f'''update saved_query 
            set CREATED_AT = TO_DATE('{random_time}', 'YYYY-MM-DD HH24:MI:SS'),
            CREATING_TIME = {random.randint(35,75)}
            where saved_query_id = {i}'''
    cur.execute(sql)

conn.commit()

In [5]:
all_query_info, latest_query_count, latest_avg_query_time = all_query_count(conn)
cpu_use_percent = 0

# CPU 사용률
if len(all_query_info) == 1:
    cpu_use_percent = round((all_query_info.iloc[-1]['CPU_CS'] - 0) / 3600, 2) * 100
else:
    cpu_use_percent = round((all_query_info.iloc[-1]['CPU_CS'] - all_query_info.iloc[-2]['CPU_CS']) / 3600, 2) * 100

In [7]:
all_query_info.tail(5)

,NOWTIME,LAST_RUN_TIME,QUERY_COUNT,AVG_CREATING_TIME,CPU_CS,TIME_QUERY
19,19,2026-02-20 19:00:00,2658,25,1619.0,100
20,20,2026-02-20 20:00:00,2743,25,1321.0,85
21,21,2026-02-20 21:00:00,2818,25,1841.0,75
22,22,2026-02-20 22:00:00,2883,38,333.0,65
23,23,2026-02-20 23:00:00,2923,25,1238.0,40


In [27]:
round(all_query_info['TIME_QUERY'].iloc[-1] / all_query_info['QUERY_COUNT'].iloc[-1] * 100,2)

np.float64(1.37)

In [3]:
user_df = pd.read_sql('select user_seq from login_session', conn)
user_seq = list(user_df['USER_SEQ'])

C:\Users\amysm\AppData\Local\Temp\ipykernel_26016\2071812375.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  user_df = pd.read_sql('select user_seq from login_session', conn)


In [4]:
import random
from datetime import datetime, timedelta

def random_datetime_today():
    now = datetime.now()
    today_start = datetime(now.year, now.month, now.day)

    # 0 ~ 86399초 중 랜덤
    random_seconds = random.randint(0, 86399)

    return today_start + timedelta(seconds=random_seconds)

random_datetime_today()

datetime.datetime(2026, 2, 19, 17, 13, 3)

In [6]:
def save_user_question(cursor, conn, CHAT_SESSION_ID, QUESTION_TIME, QUESTION_TEXT):
    sql = """
    INSERT INTO CHAT_HISTORY (CHAT_SESSION_ID, QUESTION_TIME, QUESTION_TEXT)
    VALUES (:CHAT_SESSION_ID, :QUESTION_TIME, :QUESTION_TEXT)
    """
    try:
        cursor.execute(
            sql,
            {
                "CHAT_SESSION_ID": CHAT_SESSION_ID,
                "QUESTION_TIME" : QUESTION_TIME,
                "QUESTION_TEXT": QUESTION_TEXT
            }
        )
        conn.commit()
    except Exception as e:
        conn.rollback()
        conn.commit()

def save_llm_answer(cursor, conn, CHAT_SESSION_ID, ANSWER_TIME, ANSWER_TEXT):
    QUESTION_ID = cursor.var(int)

    sql = """
    INSERT INTO CHAT_HISTORY (CHAT_SESSION_ID, ANSWER_TIME, ANSWER_TEXT)
    VALUES (:CHAT_SESSION_ID, :ANSWER_TIME, :ANSWER_TEXT)
    RETURNING QUESTION_ID INTO :QUESTION_ID
    """

    try:
        cursor.execute(
            sql,
            {
                "CHAT_SESSION_ID": CHAT_SESSION_ID,
                "ANSWER_TEXT": ANSWER_TEXT,
                "ANSWER_TIME" : ANSWER_TIME,
                "QUESTION_ID": QUESTION_ID,
            }
        )
        conn.commit()
        return QUESTION_ID.getvalue()[0]
    except Exception as e:
        conn.rollback()
        raise e

def save_sql(cursor, conn, USER_SEQ, CHAT_SESSION_ID, QUESTION_ID, SQL_TEXT, BIND_VALUES, CREATING_TIME, CREATED_AT):
    
    sql = """
    INSERT INTO SAVED_QUERY (USER_SEQ, CHAT_SESSION_ID, QUESTION_ID, SQL_TEXT, BIND_VALUES, CREATING_TIME, CREATED_AT)
    VALUES (:USER_SEQ, :CHAT_SESSION_ID, :QUESTION_ID, :SQL_TEXT, :BIND_VALUES, :CREATING_TIME, :CREATED_AT)
    """
    try:
        cursor.execute(
            sql,
            {
                "USER_SEQ" : USER_SEQ, 
                "CHAT_SESSION_ID" : CHAT_SESSION_ID, 
                "QUESTION_ID" : QUESTION_ID, 
                "SQL_TEXT" : SQL_TEXT,
                "BIND_VALUES" : BIND_VALUES,
                "CREATING_TIME" : CREATING_TIME,
                "CREATED_AT" : CREATED_AT
            }
        )
        conn.commit()
    except Exception as e:
        conn.rollback()
        raise e

In [43]:
workload_index = [
    0, 40, 38, 35, 40, 55,
    75, 110, 150 * 2, 185, 200, 195,
    170, 185, 190, 180, 160, 140 * 2,
    120, 100, 85, 75, 65, 40
]

# 마지막 쿼리 개수 55
# 15개만 30초 넘어가는 쿼리

for i in range(len(workload_index)):
    print(i)
    # 유저 질문 저장하기
    for j in range(workload_index[i]):
        USER_SEQ = random.choice(user_seq)
        CHAT_SESSION_ID = f"{USER_SEQ}_{uuid.uuid4()}"
        today = datetime.now().date()
        QUESTION_TIME = datetime.combine(today, datetime.min.time()) + timedelta(hours=i)
        save_user_question(cur, conn, CHAT_SESSION_ID, QUESTION_TIME, "이거슨 사용자 질문")
        # 답변
        QUESTION_ID = save_llm_answer(cur, conn, CHAT_SESSION_ID, QUESTION_TIME, "이거슨 llm 답변")
        save_sql(cur, conn, USER_SEQ, CHAT_SESSION_ID, QUESTION_ID, "select * from patients fetch 20 rows only", "{}", 25, QUESTION_TIME)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23


In [7]:
# hour = datetime.now().hour

# if hour >= 0:
#     all_query_info, latest_query_count, latest_avg_query_time = all_query_count(conn)
#     cpu_use_percent = 0

#     # CPU 사용률
#     if len(all_query_info) == 1:
#         cpu_use_percent = round((all_query_info.iloc[-1]['CPU_CS'] - 0) / 3600, 2) * 100
#     else:
#         cpu_use_percent = round((all_query_info.iloc[-1]['CPU_CS'] - all_query_info.iloc[-2]['CPU_CS']) / 3600, 2) * 100

#     query_graph_html = return_query_line_chart(all_query_info, 'NOWTIME', 'QUERY_COUNT')

In [44]:
df = pd.read_sql('select * from ALL_QUERY_COUNT',conn)

def save_chat_session(cursor, conn, NOWTIME, LAST_RUN_TIME, QUERY_COUNT, AVG_CREATING_TIME, TIME_QUERY):
    sql = """
        INSERT INTO ALL_QUERY_COUNT (NOWTIME, LAST_RUN_TIME, QUERY_COUNT, AVG_CREATING_TIME, CPU_CS, TIME_QUERY)
        VALUES (:NOWTIME, :LAST_RUN_TIME, :QUERY_COUNT, :AVG_CREATING_TIME, :CPU_CS, :TIME_QUERY)
    """

    try:
        cursor.execute(
            sql,
            {
                "NOWTIME": NOWTIME,
                "LAST_RUN_TIME": LAST_RUN_TIME,
                "QUERY_COUNT" : QUERY_COUNT,
                "AVG_CREATING_TIME" : AVG_CREATING_TIME,
                "CPU_CS" : random.randint(300,2000),
                "TIME_QUERY" : TIME_QUERY
            }
        )
        conn.commit()
    except Exception as e:
        conn.rollback()
        print("❌ DB Error 발생:", e)
        raise

from datetime import datetime, timedelta

today = datetime.now().date()

query = [0]

for i in range(0, 24):

    # 오늘 i시 생성
    current_timestamp = datetime.combine(today, datetime.min.time()) + timedelta(hours=i)

    sql = """
        SELECT 
            COUNT(*) AS query_count,
            ROUND(AVG(CREATING_TIME), 0) AS avg_creating_time
        FROM SAVED_QUERY
        WHERE CREATED_AT <= :CURRENT_TIME
    """

    df = pd.read_sql(sql, conn, params={'CURRENT_TIME' : current_timestamp}).to_dict(orient='records')
    query_count = int(df[0]['QUERY_COUNT']) if df[0]['QUERY_COUNT'] is not None else 0
    avg_time = int(df[0]['AVG_CREATING_TIME']) if df[0]['AVG_CREATING_TIME'] is not None else 0

    time_query = query_count - query[-1]
    query.append(time_query)

    save_chat_session(cur, conn, i, current_timestamp, query_count, avg_time, time_query)

C:\Users\amysm\AppData\Local\Temp\ipykernel_26016\2368336987.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('select * from ALL_QUERY_COUNT',conn)
C:\Users\amysm\AppData\Local\Temp\ipykernel_26016\2368336987.py:46: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conn, params={'CURRENT_TIME' : current_timestamp}).to_dict(orient='records')
C:\Users\amysm\AppData\Local\Temp\ipykernel_26016\2368336987.py:46: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conn, params

In [31]:
prev_query_count = 0

for i in range(0, 24):

    current_timestamp = datetime.combine(today, datetime.min.time()) + timedelta(hours=i)

    sql = """
        SELECT 
            COUNT(*) AS query_count,
            ROUND(AVG(CREATING_TIME), 0) AS avg_creating_time
        FROM SAVED_QUERY
        WHERE CREATED_AT <= :CURRENT_TIME
    """

    df = pd.read_sql(sql, conn, params={'CURRENT_TIME': current_timestamp}).to_dict(orient='records')

    query_count = int(df[0]['QUERY_COUNT']) if df[0]['QUERY_COUNT'] is not None else 0
    avg_time = int(df[0]['AVG_CREATING_TIME']) if df[0]['AVG_CREATING_TIME'] is not None else 0

    # 🔥 올바른 시간당 쿼리 계산
    time_query = query_count - prev_query_count

    prev_query_count = query_count

    save_chat_session(cur, conn, i, current_timestamp, query_count, avg_time, time_query)

NameError: name 'today' is not defined